In [185]:
from unittest import result
import pandas as pd
from moneyed import Money, EUR


# debugger
def debug(print_statement):
    print(print_statement)


"""_summary_
Neemt cursusinformatie en geeft een dataframe waarin de omzet lineair per week verdeeld
wordt. Houdt daarbij rekening met gegeven jaaromzetten. Als een jaaromzet gegeven is,
wordt die omzet binnen het jaar lineair verdeeld.
"""


def calc_omzet_per_week(row) -> pd.DataFrame:
    assert "Cursuscode" in row.index
    assert "Startdatum" in row.index
    assert "Einddatum" in row.index
    assert "Omzet" in row.index

    debug(f"############")
    debug(f"Start berekening voor {row['Cursuscode']}")

    # dit zijn alle omzetweken in totaal (als datum datatype)
    daterange = pd.date_range(
        row["Startdatum"], row["Einddatum"], freq="W", inclusive="both"
    )

    # voor de voorbeeldberekening is een vast aantal vakantieweken opgenomen
    # in een productieversie kan gekozen worden om deze data te ontlenen aan
    # de API van de rijksoverheid.
    vakantieweken = [1, 8, 18, 19, 29, 30, 31, 32, 33, 34, 51, 52]
    weken_zonder_vakantie = []
    lesweken_per_jaar = {}

    # alle weken met een jaartal dat voorkomt in de gegeven data worden apart genomen.
    # de rest van de weken wordt onder de sleutel 'rest' opgenomen.
    for week in daterange:
        if week.isocalendar().week in vakantieweken:
            continue
        sleutel = "resterend"
        if week.year in row.index:
            sleutel = week.year
        if sleutel in lesweken_per_jaar:
            lesweken_per_jaar[sleutel].append(week)
        else:
            lesweken_per_jaar[sleutel] = [week]

    df_result = pd.DataFrame(columns=weken_zonder_vakantie)

    # we gaan per jaar de omzet per week berekenen
    # als voor een jaar een omzetbedrag gegeven is, wordt die omzet lineair
    # verdeeld over de werkweken van dat jaar
    restomzet = Money(row["Omzet"], EUR)  # de totaal resterende, te verdelen omzet
    debug(f"Restomzet bij aanvang van de berekening is {restomzet}")
    for jaar, weken in lesweken_per_jaar.items():
        jaaromzet = Money(0, EUR)
        weekomzet = Money(0, EUR)
        aantal_lesweken = len(lesweken_per_jaar[jaar])
        if jaar in row.index:  # is er een gegeven omzet voor dit jaar?
            jaaromzet = Money(row[jaar], EUR)
            weekomzet = (
                jaaromzet / aantal_lesweken
            )  # wat is de weekomzet voor dit jaar?
            restomzet -= jaaromzet
            debug(f"--- Berkening voor {jaar} ---")
            debug(f"Jaar {jaar} heeft {weken[0]} als eerste datum.")
            debug(f"Jaar {jaar} heeft {weken[-1]} als laatste datum.")
            debug(
                f"Jaar {jaar} heeft {jaaromzet} als jaaromzet met weekomzet {weekomzet} in {aantal_lesweken} werkweken."
            )
            debug(f"Jaar {jaar} eindigt met restomzet {restomzet}")
        else:  # er is geen gegeven omzet voor dit jaar
            weekomzet = (
                restomzet / aantal_lesweken
            )  # wat is de weekomzet voor dit jaar?
            debug(
                f"Geen volgende jaaromzet gevonden. Resterende omzet is {restomzet} in {aantal_lesweken} werkweken. Weekomzet is {weekomzet}"
            )

        for week in weken:
            newrow = {
                "Jaar": week.year,
                "Maand": week.month,
                "Week": week,
                "Omzet": weekomzet,
            }
            df_result = pd.concat(
                [df_result, pd.DataFrame.from_dict([newrow])], ignore_index=True
            )
    df_result.insert(0, "Cursuscode", row["Cursuscode"])

    # elementaire controles
    input_totale_omzet = Money(row["Omzet"], EUR)
    result_totale_omzet = df_result["Omzet"].sum()
    debug(
        f"in: {input_totale_omzet.amount} ({type(input_totale_omzet)}), uit: {result_totale_omzet.amount} ({type(result_totale_omzet)})"
    )
    controle_decimalen = 5
    assert round(result_totale_omzet.amount, controle_decimalen) == round(
        input_totale_omzet.amount, 5
    )  # omzet: som output gelijk aan input?
    debug(
        f"Omzetberekening voor {row['Cursuscode']} correct tot {controle_decimalen} posities achter de komma."
    )

    return df_result

## Een voorbeeldberekening
De testdata is wijzigbaar, probeer dus verschillende dingen uit!

In [186]:
# voorbeelddata
testdata = [
    {
        "Cursuscode": "KP22A",
        "Startdatum": pd.to_datetime("2022-04-01"),
        "Einddatum": pd.to_datetime("2026-02-15"),
        "Omzet": 500_000,
        2022: 100_000,
        2023: 150_000,
    }
]
df = pd.DataFrame(testdata)

result = calc_omzet_per_week(df.iloc[0, :])

############
Start berekening voor KP22A
Restomzet bij aanvang van de berekening is €500,000.00
--- Berkening voor 2022 ---
Jaar 2022 heeft 2022-04-03 00:00:00 als eerste datum.
Jaar 2022 heeft 2022-12-18 00:00:00 als laatste datum.
Jaar 2022 heeft €100,000.00 als jaaromzet met weekomzet €3,333.33 in 30 werkweken.
Jaar 2022 eindigt met restomzet €400,000.00
--- Berkening voor 2023 ---
Jaar 2023 heeft 2023-01-15 00:00:00 als eerste datum.
Jaar 2023 heeft 2023-12-17 00:00:00 als laatste datum.
Jaar 2023 heeft €150,000.00 als jaaromzet met weekomzet €3,750.00 in 40 werkweken.
Jaar 2023 eindigt met restomzet €250,000.00
Geen volgende jaaromzet gevonden. Resterende omzet is €250,000.00 in 86 werkweken. Weekomzet is €2,906.98
in: 500000 (<class 'moneyed.classes.Money'>), uit: 500000.0000000000000000000019 (<class 'moneyed.classes.Money'>)
Omzetberekening voor KP22A correct tot 5 posities achter de komma.


## Resultaat
Het resultaat van de voorbeeldbereking in transpositie, zodat de opeenvolgende weken in kolommen staan.

In [187]:
result.set_index(result["Week"])[["Omzet"]].transpose()

Week,2022-04-03,2022-04-10,2022-04-17,2022-04-24,2022-05-01,2022-05-22,2022-05-29,2022-06-05,2022-06-12,2022-06-19,...,2025-11-23,2025-11-30,2025-12-07,2025-12-14,2026-01-11,2026-01-18,2026-01-25,2026-02-01,2026-02-08,2026-02-15
Omzet,"€3,333.33","€3,333.33","€3,333.33","€3,333.33","€3,333.33","€3,333.33","€3,333.33","€3,333.33","€3,333.33","€3,333.33",...,"€2,906.98","€2,906.98","€2,906.98","€2,906.98","€2,906.98","€2,906.98","€2,906.98","€2,906.98","€2,906.98","€2,906.98"


Het resultaat van de voorbeelberekning in transpositie, verdicht naar jaren en maanden.

In [188]:
result[["Jaar", "Maand", "Omzet"]].groupby(["Jaar", "Maand"]).sum().transpose()

Jaar         2022                                                              \
Maand          4           5           6           7           9           10   
Omzet  €13,333.33  €10,000.00  €13,333.33  €10,000.00  €13,333.33  €16,666.67   

Jaar                                 2023              ...       2025  \
Maand          11          12          1           2   ...         5    
Omzet  €13,333.33  €10,000.00  €11,250.00  €11,250.00  ...  €5,813.95   

Jaar                                                                         \
Maand          6          7          8           9           10          11   
Omzet  €14,534.88  €5,813.95  €2,906.98  €11,627.91  €11,627.91  €14,534.88   

Jaar                   2026             
Maand         12         1          2   
Omzet  €5,813.95  €8,720.93  €8,720.93  

[1 rows x 44 columns]

Het resultaat van de voorbeelberekning in transpositie, verdicht naar jaren.

In [189]:
result[["Jaar", "Omzet"]].groupby("Jaar").sum().transpose()

Jaar,2022,2023,2024,2025,2026
Omzet,"€100,000.00","€150,000.00","€116,279.07","€116,279.07","€17,441.86"
